In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft
import threading
import socket
import time
import zlib
import pyodbc

In [ ]:
def tcplink(sock,addr):
    print('Accept new connection from %s:%s...'%addr)
    sock.send(b'Server Connected!')
    total_data=''
    data=''
    temp=[]
    while True:
        data = sock.recv(1024) #set buffer for rcving data
        if not data data.decode('utf-8')=='end':   #if no data rcved or 'end' rcved,break the loop
            break
        total_data=total_data+data.decode('utf-8')      #decode the data rcved and integrate into astring
        if data[-4:-1].decode('utf-8')=='end':     #if a string contain 'end', means all data rcved
            break
    temp_raw_data=total_data.split('ssss')  #the data rcved are generally separated by the'ssss'
    cmd=temp_raw_data[0]    #the first segment of data rcved contain the command information, like FFTC, FFT calculation requested etc.
    print(type(cmd),cmd)    #used for commissioning
    if cmd=='FFTC':     #means a FFT calculation requested
        temp=temp_raw_data[1]   #according to the protocol defined, the No.2 element contain Frequence information
        N=int(temp)     #transfer a string Frequence into INT
        print(N)        #used for commissioning
        temp_MV_value=temp_raw_data[2].split('\n')[0:N]     #Get a actrual measured value list of length N
        
        T = 1.0 / N    # get a sample spacing
        x = np.linspace(0.0, 1.0, N)    #arrange axe values
        y=list(map(float,temp_MV_value))   #transfer to float from string according to the sample rate
        yf = fft(y)    #calculate FFT
        
        yf[0]=0     #set the DC part to zero(DC filted)
        
        xf = np.linspace(0.0, 1.0/(2.0*T), int(N/2))    #arrange axe values for FFT drawing, only half inputted frequence arranged
        
        ZOOM1=4  #define the length of axe of N/ZOOM1
        fig, ax = plt.subplots(2, 1,figsize=(10,10))    #define type, size of plot
        ax[0].plot(x[0:int(N/ZOOM1)],y[0:int(N/ZOOM1)],'g')     #the first one show the measured values curve
        ax[0].set_xlabel('Time')
        ax[0].set_ylabel('Amplitude')

        ZOOM2=8  #define the length of axe of N/ZOOM2
        ax[1].plot(xf[0:int(N/ZOOM2)],(2.0/N)* np.abs(yf[0:int(N/ZOOM2)]),'r') # plotting the spectrum
        ax[1].set_xlabel('Freq (Hz)')
        ax[1].set_ylabel('|Y(freq)|')
    
        plt.grid()
        plt.show()
    
        temp_str_yf='' 
        #get a string contain the information of each frequence and spectrum of each frequence
        for index in range(0,len(yf)):
            temp=str(index)+':'+str(int((2.0/N)* np.abs(yf[index])*1000)/1000)+'\n' #only 4 decimal places reserved
            temp_str_yf=temp_str_yf+temp

        print(len(temp_str_yf.encode('utf-8')))     #used for commissioning
        temp_str_yf_zip=zlib.compress(temp_str_yf.encode('utf-8'))  #encoded to a zip

        sock.send(zlib.decompress(temp_str_yf_zip))     #used for commissioning
        print(len(temp_str_yf_zip))      #used for commissioning
    
        time.sleep(2) 

        #Connect to SQL Server and save the data, Open database connection
        db=pyodbc.connect('DRIVER={SQL Server};SERVER=SHHJX-9928EB342;DATABASE=das;UID=sa;PWD=200942')
        cursor = db.cursor()        # prepare a cursor object using cursor() method
        time1=time.strftime('%Y-%m-%d:%H-%M-%S',time.localtime(time.time()))    #get date & time
        query="insert into FFT_test (datetime,Frequence,Measured_value,FFTed_value) values ('"+time1+"','"+str(N)+"','"+temp_raw_data[2]+"','"+temp_str_yf+"')"
        cursor.execute(query)   #write date & time, Frequence and actrual measured value into SQL Server
        cursor.commit()     #submit a query

        db.close()  #close the connection to SQL Server

    else:
        print('Unknow command.')    #coming Next...
    sock.close()    #close the socket
    print('Connection from %s:%s Closed.'%addr)  #show the message

In [ ]:
s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)  #creat a object of TCP socket
s.bind(('192.168.1.185',5555))  #bind to a IP address and a COM port 5555
s.listen(10)    #ready to rcv message from client
print('Waiting for connection...')  #show the message

In [ ]:
try:
#    thread_list=[]      #used for commissioning
    while True:
        sock, addr = s.accept()     #get the client sock and IP address when a client ask for connection
        t = threading.Thread(target=tcplink, args=(sock, addr)) #define a thread object named t.
#        thread_list.append(t)   #used for commissioning, append to list when a thread requested
#        print(thread_list)      #used for commissioning
        t.setDaemon(True)   #used for killing a thread
        t.start()       #start a thread
    
except KeyboardInterrupt:
    print ('You stop the threading')    #kill a thread by preeing Ctrl-Z

print ('%s thread end!'%(time.ctime())) #show the message